The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

In [1]:
!pip install turicreate

     |████████████████████████████████| 92.0 MB 1.3 MB/s 
     |████████████████████████████████| 322 kB 35.0 MB/s 
     |████████████████████████████████| 86.4 MB 65 kB/s 
     |████████████████████████████████| 3.5 MB 37.7 MB/s 
     |████████████████████████████████| 3.6 MB 37.7 MB/s 
     |████████████████████████████████| 18.3 MB 400 kB/s 
     |████████████████████████████████| 20.1 MB 1.2 MB/s 
     |████████████████████████████████| 3.8 MB 39.7 MB/s 
     |████████████████████████████████| 2.9 MB 34.1 MB/s 
     |████████████████████████████████| 50 kB 6.7 MB/s 
     |████████████████████████████████| 449 kB 11.4 MB/s 
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13714 sha256=e493ac8bec8df9a416c99387fbd5d5e9b03770918f4e74079698c5c5ac83ff3d
  Stored in directory: /root/.cache/pip/wheels/b2/7f/f6/f180315b584f00445045ff1699b550fa895d09471337ce21c6
  Created wheel for resampy: filename=resampy-0.2.1-py3-none-any.whl size=320859 sha256=2133c71c77

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
cwd_old = os.getcwd()
print('\nWorking directory: ', cwd_old)

os.chdir('/content/gdrive/MyDrive/Machine Learning - Uni of Washington')
print('Working directory has changed')

# verifying the current directory has changed
cwd_new = os.getcwd()
print('\nCurrent Working directory: ', cwd_new)


Working directory:  /content
Working directory has changed

Current Working directory:  /content/gdrive/MyDrive/Machine Learning - Uni of Washington


# Fire up Turi Create

In [3]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [4]:
sales = turicreate.SFrame('home_data.sframe')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [5]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [6]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = turicreate.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                    validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training Max Error | Training Root-Mean-Square Error |

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 1.015001     | 4146407.600631     | 258679.804477                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [7]:
example_weight_summary = example_model.coefficients
print(example_weight_summary)

+-------------+-------+--------------------+--------------------+
|     name    | index |       value        |       stderr       |
+-------------+-------+--------------------+--------------------+
| (intercept) |  None | 87910.07249239809  | 7873.338143401673  |
| sqft_living |  None |   315.4034405521   | 3.455700325854743  |
|   bedrooms  |  None | -65080.21555282686 | 2717.456854420703  |
|  bathrooms  |  None | 6944.020192636717  | 3923.1149314414993 |
+-------------+-------+--------------------+--------------------+
[4 rows x 4 columns]



# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [8]:
example_predictions = example_model.predict(train_data)
print(example_predictions[0])

271789.5058780322


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [9]:
def get_residual_sum_of_squares(model, data, outcome):
    predictions = model.predict(data)
    residuals = outcome - predictions
    RSS = (residuals * residuals).sum()
    
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [10]:
rss_example_test = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print(rss_example_test)

273761538330193.2


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [11]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 

As an example here's the first one:

In [12]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [13]:
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']

In [14]:
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

In [15]:
train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

In [16]:
train_data[['bedrooms', 'bedrooms_squared', 'bathrooms', 'bed_bath_rooms', 'sqft_living', 'log_sqft_living', 'lat', 'long', 'lat_plus_long']].head()

bedrooms,bedrooms_squared,bathrooms,bed_bath_rooms,sqft_living,log_sqft_living,lat
3.0,9.0,1.0,3.0,1180.0,7.07326971745971,47.51123398
3.0,9.0,2.25,6.75,2570.0,7.851661177889265,47.72102274
2.0,4.0,1.0,2.0,770.0,6.646390514847729,47.73792661
4.0,16.0,3.0,12.0,1960.0,7.580699752224563,47.52082
3.0,9.0,2.0,6.0,1680.0,7.426549072397305,47.61681228
4.0,16.0,4.5,18.0,5420.0,8.597851094433691,47.65611835
3.0,9.0,2.25,6.75,1715.0,7.44716835960004,47.30972002
3.0,9.0,1.5,4.5,1060.0,6.966024187106113,47.40949984
3.0,9.0,1.0,3.0,1780.0,7.484368643286131,47.51229381
3.0,9.0,2.5,7.5,1890.0,7.5443321080536885,47.36840673


In [17]:
test_data[['bedrooms', 'bedrooms_squared', 'bathrooms', 'bed_bath_rooms', 'sqft_living', 'log_sqft_living', 'lat', 'long', 'lat_plus_long']].head()

bedrooms,bedrooms_squared,bathrooms,bed_bath_rooms,sqft_living,log_sqft_living,lat
3.0,9.0,1.0,3.0,1430.0,7.265429723253953,47.75584254
4.0,16.0,3.0,12.0,2950.0,7.989560449333865,47.57136955
3.0,9.0,2.0,6.0,1710.0,7.444248649496705,47.30482931
3.0,9.0,2.5,7.5,2320.0,7.749322464660356,47.5391103
3.0,9.0,1.0,3.0,1090.0,6.993932975223189,47.68889559
4.0,16.0,2.5,10.0,2620.0,7.8709295967551425,47.53013988
4.0,16.0,2.25,9.0,4220.0,8.347590407030058,47.44504345
4.0,16.0,2.5,10.0,2250.0,7.718685495198466,47.36628767
3.0,9.0,1.75,5.25,1260.0,7.138866999945524,47.43658598
4.0,16.0,2.0,8.0,2750.0,7.919356190660617,47.7168015


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [18]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [19]:
model_1 = turicreate.linear_regression.create(train_data, target = 'price', features = model_1_features, validation_set = None)
model_2 = turicreate.linear_regression.create(train_data, target = 'price', features = model_2_features, validation_set = None)
model_3 = turicreate.linear_regression.create(train_data, target = 'price', features = model_3_features, validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 0.018575     | 3193229.177888     | 228200.043155                   |

| 2         | 3        | 0.047970     | 3193229.177873     | 228200.043155                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

In [20]:
# Examine/extract each model's coefficients:
model_1_weight_summary = model_1.coefficients
model_2_weight_summary = model_2.coefficients
model_3_weight_summary = model_3.coefficients

print(model_1_weight_summary)
print(model_2_weight_summary)
print(model_3_weight_summary)

+-------------+-------+---------------------+--------------------+
|     name    | index |        value        |       stderr       |
+-------------+-------+---------------------+--------------------+
| (intercept) |  None | -56140675.741145164 | 1649985.420232753  |
| sqft_living |  None |  310.26332577692176 | 3.188829604072599  |
|   bedrooms  |  None |  -59577.11606759663 | 2487.279773224208  |
|  bathrooms  |  None |  13811.840541653299 | 3593.5421329676874 |
|     lat     |  None |  629865.7894714857  | 13120.71003228041  |
|     long    |  None | -214790.28516471686 | 13284.285159947443 |
+-------------+-------+---------------------+--------------------+
[6 rows x 4 columns]

+----------------+-------+---------------------+--------------------+
|      name      | index |        value        |       stderr       |
+----------------+-------+---------------------+--------------------+
|  (intercept)   |  None |  -54410676.11184308 | 1650405.1653694494 |
|  sqft_living   |  None |  

# Quiz

**What is the mean (arithmetic average) value of your 4 new features on TEST data?**

In [21]:
print(test_data['bed_bath_rooms'].mean())
print(test_data['bedrooms_squared'].mean())
print(test_data['log_sqft_living'].mean())
print(test_data['lat_plus_long'].mean())

7.503901631591395
12.4466777015843
7.550274679645938
-74.65333497217313


**What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

In [22]:
a = 'Positive'
print("The sign for the coefficient for bathrooms in model 1 is:", a)

The sign for the coefficient for bathrooms in model 1 is: Positive


**What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

In [23]:
b = 'Negative'
print("The sign for the coefficient for bathrooms in model 2 is:", b)

The sign for the coefficient for bathrooms in model 2 is: Negative


**Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?**

In [24]:
rss_model_1_train_data = get_residual_sum_of_squares(model_1, train_data, train_data['price'])
rss_model_2_train_data = get_residual_sum_of_squares(model_2, train_data, train_data['price'])
rss_model_3_train_data = get_residual_sum_of_squares(model_3, train_data, train_data['price'])

print(rss_model_1_train_data)
print(rss_model_2_train_data)
print(rss_model_3_train_data)

971328233545435.4
961592067857506.0
905276314551642.5


**Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?**

In [25]:
rss_model_1_test_data = get_residual_sum_of_squares(model_1, test_data, test_data['price'])
rss_model_2_test_data = get_residual_sum_of_squares(model_2, test_data, test_data['price'])
rss_model_3_test_data = get_residual_sum_of_squares(model_3, test_data, test_data['price'])

print(rss_model_1_test_data)
print(rss_model_2_test_data)
print(rss_model_3_test_data)

226568089093160.38
224368799993971.25
251829318963159.2
